In [1]:
import pandas as pd
import numpy as np
import sys
import torch

In [2]:
df_train = pd.read_csv(r"C:\Users\maxim\Desktop\js\linfit\test\archive\mnist_train.csv")
df_test = pd.read_csv(r"C:\Users\maxim\Desktop\js\linfit\test\archive\mnist_test.csv")

In [3]:
xcolnams = df_train.columns.tolist()
print("check0")
df_train[xcolnams[1:]] /= 255. #convert the pixel values to 0..1
print("check1")

df_train["ones"] = 1.
xcolnams += ["ones"]
Xs_train = torch.tensor(df_train[xcolnams[1:]].values.T).float().cuda()
print("check2")

check0
check1
check2


In [4]:
ycolnams = []
for i in range(10):
    coln = "is"+str(i)
    ycolnams.append(coln)
    df_train[coln] = (df_train["label"] == i).astype(float)

In [5]:
xcolnams = df_test.columns.tolist()
print("check0")
df_test[xcolnams[1:]] /= 255. #convert the pixel values to 0..1
print("check1")

df_test["ones"] = 1.
xcolnams += ["ones"]
Xs_test = torch.tensor(df_test[xcolnams[1:]].values.T).float().cuda()
print("check2")

check0
check1
check2


In [6]:
ycolnams = []
for i in range(10):
    coln = "is"+str(i)
    ycolnams.append(coln)
    df_test[coln] = (df_test["label"] == i).astype(float)

In [7]:
Ys_train = torch.tensor(df_train[ycolnams + ["ones"]].values.T).float().cuda()

In [8]:
Ys_test = torch.tensor(df_test[ycolnams + ["ones"]].values.T).float().cuda()

In [9]:
sys.path.append(r"C:\Users\maxim\Desktop\js\linfit")
from torch_linfit import lay, lay_holder

In [10]:
Ys_train.shape

torch.Size([11, 60000])

In [39]:
model = lay_holder([lay(784,400), lay(400,10)]) 
#model = lay_holder([lay(784,10, False)])

In [40]:
NTR = 1
for j in range(NTR):
    print("Back")
    for i in range(len(model.layers)):
        model.solve_update(Xs_train, Ys_train, n=len(model.layers)-i)
        forw = model.forward(Xs_train)
        mse = torch.mean((forw[0,:] - Ys_train[0,:])**2)
        print(mse)

    print("Front")
    for i in range(len(model.layers)):
        model.solve_update(Xs_train, Ys_train, n=i + 1)
        forw = model.forward(Xs_train)
        #print(forw.shape)
        mse = torch.mean((forw[0,:] - Ys_train[0,:])**2)
        print(mse)

Back
tensor(0.0221, device='cuda:0')
tensor(0.1866, device='cuda:0')
Front
tensor(0.0429, device='cuda:0')
tensor(0.0108, device='cuda:0')


In [41]:
forw = model.forward(Xs_train)
# #print(forw.shape)
# mse = np.mean((forw[0,:] - Ys_train[0,:])**2)

guessed = torch.argmax(forw[:-1,:], axis=0).cpu().numpy()
# print(mse)

In [42]:
guessed

array([5, 0, 4, ..., 5, 6, 8], dtype=int64)

In [43]:
sum(guessed == df_train["label"].values) / len(guessed)

0.8946666666666667

In [44]:
1/10

0.1

In [45]:
from sklearn.metrics import confusion_matrix

In [46]:
confusion_matrix(df_train["label"].values, guessed)

array([[5713,    2,   36,   21,    9,   32,   44,   10,   53,    3],
       [   1, 6431,   59,   18,   24,   23,    5,   31,  141,    9],
       [  80,   54, 5252,  114,   97,   18,  122,   64,  126,   31],
       [  25,   50,  199, 5162,   18,  313,   23,  104,  175,   62],
       [  10,   34,   57,    8, 5335,   24,   60,   13,   43,  258],
       [  78,   20,   58,  296,   36, 4474,  106,   53,  252,   48],
       [  56,   16,   71,   11,   38,  107, 5574,    4,   37,    4],
       [  25,   93,   73,   27,  108,   26,    3, 5712,   11,  187],
       [  35,  240,   83,  202,   62,  211,   53,   20, 4853,   92],
       [  38,   29,   40,   94,  256,   52,    5,  195,   66, 5174]],
      dtype=int64)

In [47]:
forw = model.forward(Xs_test)
# #print(forw.shape)
# mse = np.mean((forw[0,:] - Ys_train[0,:])**2)

guessed = torch.argmax(forw[:-1,:], axis=0).cpu().numpy()
# print(mse)

In [48]:
sum(guessed == df_test["label"].values) / len(guessed)

0.8935

In [49]:
confusion_matrix(df_test["label"].values, guessed)

array([[ 956,    0,    2,    1,    0,    5,   10,    1,    5,    0],
       [   0, 1098,    4,    4,    1,    1,    3,    3,   21,    0],
       [  13,    7,  898,   23,   20,    2,   25,   13,   26,    5],
       [   4,    4,   24,  862,    3,   56,    1,   19,   28,    9],
       [   1,    6,    8,    0,  901,    1,   12,    4,   10,   39],
       [  13,    3,    6,   47,    9,  739,   13,   14,   43,    5],
       [  14,    4,   11,    1,    9,   20,  891,    1,    6,    1],
       [   2,   18,   20,    7,   12,    0,    1,  918,    4,   46],
       [  12,   24,    8,   42,   19,   36,   13,    5,  804,   11],
       [  10,    8,    4,   10,   51,    7,    0,   34,   17,  868]],
      dtype=int64)